In [1]:
#Imports
import pandas as pd
import os
import tensorflow as tf

import sys  
sys.path.append("../../../")  
from utils.models import *
from utils.datahandling import *
from utils.modelrunner import *

import wandb
import logging
logging.getLogger("wandb").setLevel(logging.ERROR)
logging.getLogger('tensorflow').setLevel(logging.ERROR)

os.environ['WANDB_SILENT'] = 'true'
os.environ['WANDB_CONSOLE'] = 'off'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Data processing

In [2]:
#Get data 
num_users = 30

cwd = os.path.normpath(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))))
df = pd.read_csv(cwd+'/data/3final_data/Final_Totalload_dataset.csv', index_col='Date')
df.index = pd.to_datetime(df.index)
df.fillna(0, inplace=True)

# Get the first date from the index
start_date = df.index.min()
# Calculate the end date as one year from the start date
end_date = start_date + pd.DateOffset(years=1)
# Filter the dataframe to only include the first year of data
df = df[(df.index >= start_date) & (df.index < end_date)]

df_array = []
for idx in range(num_users):
    df_array.append(df[[f'User{idx+1}', 'temp', 'rhum', 'wspd', 'PC1', 'hour sin', 'hour cos', f'User{idx+1}_lag_24hrs']])

df_array[0].head(1)

,User1,temp,rhum,wspd,PC1,hour sin,hour cos,User1_lag_24hrs
Date,,,,,,,,
2010-07-02,1.199,2.5,92.0,0.0,-2.641741,0.0,1.0,1.2


In [3]:
#Hyperparameters
sequence_length = 25
batch_size = 16
num_features = df_array[0].shape[1]
horizon = 1
max_epochs = 100

loss = tf.keras.losses.MeanSquaredError()
metrics=[
    tf.keras.metrics.RootMeanSquaredError(), 
    tf.keras.metrics.MeanAbsolutePercentageError(),
    tf.keras.metrics.MeanAbsoluteError(),
]

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,mode='min')
timing_callback = TimingCallback()
custom_callback = CustomCallback()

callbacks=[early_stopping, timing_callback, custom_callback]

#BiLSTM
lstm_layers = 2
lstm_units = 8
lstm_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])
lstm_results = pd.DataFrame(columns=['architecture', 'train_time', 'avg_time_epoch', 'mse','mse_std', 'rmse','rmse_std','mape','mape_std','mae','mae_std'])

#CNN
cnn_layers = 4
cnn_kernel_size = 1
cnn_filter_size = 8
cnn_dense_units = 16
cnn_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])
cnn_results = pd.DataFrame(columns=['architecture', 'train_time', 'avg_time_epoch', 'mse','mse_std', 'rmse','rmse_std','mape','mape_std','mae','mae_std'])

#Transfotransformer_architecture = "Transformer_ED2_h4_d32"
transformer_layers = 2
transformer_heads = 4
transformer_dense_units = 16
transformer_all_results = pd.DataFrame(columns=["user", "architecture", "train_time", "avg_time_epoch", "mse", "rmse", "mape", "mae"])
transformer_results = pd.DataFrame(columns=['architecture', 'train_time', 'avg_time_epoch', 'mse','mse_std', 'rmse','rmse_std','mape','mape_std','mae','mae_std'])

In [4]:
#Train, Validation and Test datasets
X_train, y_train, X_val, y_val, X_test, y_test = {}, {}, {}, {}, {}, {}

#Create Train, Validation and Test datasets
for idx, df in enumerate(df_array):
    n = len(df)
    train_df = df[0:int(n*0.7)]
    val_df = df[int(n*0.7):int(n*0.9)]
    test_df = df[int(n*0.9):]

    # Min max sclaing
    train_df = min_max_scaling(train_df)
    val_df = min_max_scaling(val_df)
    test_df = min_max_scaling(test_df)

    # Sequencing
    train_sequences = create_sequences(train_df, sequence_length)
    val_sequences = create_sequences(val_df, sequence_length)
    test_sequences = create_sequences(test_df, sequence_length)

    #Split into feature and label
    X_train[f'user{idx+1}'], y_train[f'user{idx+1}'] = prepare_data(train_sequences, batch_size)
    X_val[f'user{idx+1}'], y_val[f'user{idx+1}'] = prepare_data(val_sequences, batch_size)
    X_test[f'user{idx+1}'], y_test[f'user{idx+1}'] = prepare_data(test_sequences, batch_size)

### Benchmark models

In [5]:
run_bilstm_model(
    wb_project_name = "TS_LL_Totalload_Forecasting_Benchmark",
    wb_model_name = "bilstm",
    wb_project = "TS_LL_Totalload",
    save_path = os.getcwd(),
    df_array = df_array,
    max_epochs = max_epochs,
    batch_size = batch_size,
    X_train = X_train,
    horizon = horizon, 
    lstm_layers = lstm_layers, 
    lstm_units = lstm_units,    
    metrics = metrics,
    loss = loss,
    y_train = y_train,
    X_val = X_val,
    y_val = y_val,
    X_test = X_test,
    y_test = y_test,
    callbacks = callbacks,
    results = lstm_results,
    all_results = lstm_all_results
)

-----User:  1
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  2
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  3
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  4
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  5
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  6
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  7
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  8
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----Use

In [6]:
run_cnn_model(
    wb_project_name = "TS_LL_Totalload_Forecasting_Benchmark",
    wb_model_name = "cnn",
    wb_project = "TS_LL_Totalload",
    save_path = os.getcwd(),
    df_array = df_array,
    max_epochs = max_epochs,
    batch_size = batch_size,
    X_train = X_train,
    horizon = horizon, 
    cnn_layers = cnn_layers, 
    cnn_filter_size = cnn_filter_size,
    cnn_kernel_size = cnn_kernel_size, 
    cnn_dense_units = cnn_dense_units,      
    metrics = metrics,
    loss = loss,
    y_train = y_train,
    X_val = X_val,
    y_val = y_val,
    X_test = X_test,
    y_test = y_test,
    callbacks = callbacks,
    results = cnn_results,
    all_results = cnn_all_results
)

-----User:  1
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  2
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  3
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  4
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  5
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  6
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  7
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  8
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----Use

In [7]:
run_transformer_model(
     wb_project_name = "TS_LL_Totalload_Forecasting_Benchmark",
    wb_model_name = "transformer",
    wb_project = "TS_LL_Totalload",
    save_path = os.getcwd(),
    df_array = df_array,
    max_epochs = max_epochs,
    batch_size = batch_size,
    X_train = X_train,
    horizon = horizon, 
    sequence_length = sequence_length, 
    transformer_layers = transformer_layers, 
    num_features = num_features, 
    transformer_heads = transformer_heads, 
    transformer_dense_units = transformer_dense_units,  
    metrics = metrics,
    loss = loss,
    y_train = y_train,
    X_val = X_val,
    y_val = y_val,
    X_test = X_test,
    y_test = y_test,
    callbacks = callbacks,
    results = transformer_results,
    all_results = transformer_all_results
)

-----User:  1
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  2
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  3
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  4
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  5
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  6
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  7
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----User:  8
Round:  0
saved model
Round:  1
saved model
Round:  2
saved model
Round:  3
saved model
Round:  4
saved model
-----Use